# 02 - Data Processing

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import numpy as np
import pandas as pd

import stock_utils.constants as cnst
import stock_utils.data as sd

pd.set_option('display.max_columns', None)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
SCREENER_EPS_DATA_DIR = PosixPath('../data/Screener.in/EPS') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Constants

In [3]:
stock_symbols = sd.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['DEEPAKFERT', 'HDFCBANK', 'INDIGOPNTS', 'ITBEES', 'JUBLFOOD', 'NH']

In [4]:
STOCK_SYMBOL = stock_symbols[1]
STOCK_SYMBOL

'HDFCBANK'

## Data loading

In [5]:
stock_df = pd.read_parquet(cnst.NSE_DATA_DIR.joinpath(STOCK_SYMBOL, "consolidated.parquet"))
stock_df.columns = [
    'Date', 'Series',
    'Open', 'High', 'Low', 'Prev Close', 'LTP', 'Close',
    'VWAP', '52W H', '52W L',
    'Volume', 'Value', '#Trades'
] + (["PE"] if "PE" in stock_df.columns else [])
stock_df

,Date,Series,Open,High,Low,Prev Close,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE
0,2020-01-01,EQ,1276.10,1280.00,1270.60,1272.10,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,31.185
1,2020-01-02,EQ,1279.00,1288.00,1279.00,1278.60,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,31.384
2,2020-01-03,EQ,1282.20,1285.00,1263.60,1286.75,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,30.937
3,2020-01-06,EQ,1260.00,1261.80,1236.00,1268.40,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,30.267
4,2020-01-07,EQ,1258.90,1271.45,1252.25,1240.95,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,30.746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,2024-05-03,EQ,1533.25,1540.60,1507.20,1532.25,1515.65,1519.60,1521.12,1757.5,1363.55,15932621,2.423550e+10,309946,18.433
1093,2024-05-06,EQ,1522.00,1534.50,1517.15,1519.60,1523.75,1522.65,1525.73,1757.5,1363.55,14396698,2.196542e+10,290222,18.470
1094,2024-05-07,EQ,1521.95,1524.05,1504.15,1522.65,1505.10,1506.15,1510.70,1757.5,1363.55,14240301,2.151282e+10,344750,18.270
1095,2024-05-08,EQ,1495.95,1496.95,1480.35,1506.15,1483.50,1482.65,1486.59,1757.5,1363.55,20843931,3.098642e+10,465848,17.985


In [6]:
stock_df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1097 entries, 0 to 1096
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        1097 non-null   datetime64[ns]
 1   Series      1097 non-null   object        
 2   Open        1097 non-null   float64       
 3   High        1097 non-null   float64       
 4   Low         1097 non-null   float64       
 5   Prev Close  1097 non-null   float64       
 6   LTP         1097 non-null   float64       
 7   Close       1097 non-null   float64       
 8   VWAP        1097 non-null   float64       
 9   52W H       1097 non-null   float64       
 10  52W L       1097 non-null   float64       
 11  Volume      1097 non-null   int64         
 12  Value       1097 non-null   float64       
 13  #Trades     1097 non-null   int64         
 14  PE          1097 non-null   float64       
dtypes: datetime64[ns](1), float64(11), int64(2), object(1)
memory usage: 183

## Feature engineering

### Daily candle based columns

In [7]:
stock_df['Range'] = stock_df['High'] - stock_df['Low']
stock_df['IsGreen'] = (stock_df['Close'] >= stock_df['Prev Close']).astype(int)
stock_df['Is52WLow'] = np.isclose(stock_df['Low'], stock_df['52W L'], atol = 0, rtol = 1e-4).astype(int)
stock_df['Is52WHigh'] = np.isclose(stock_df['High'], stock_df['52W H'], atol = 0, rtol = 1e-4).astype(int)
stock_df = stock_df.drop(columns = ['Series', 'Prev Close'])
stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh
196,2020-09-21,1055.0,1071.35,1041.5,1042.10,1049.30,1058.70,1305.50,738.75,9921439,1.050383e+10,206109,21.113,29.85,0,0,0
168,2020-08-12,1065.0,1066.00,1048.0,1063.00,1063.70,1057.73,2288.80,738.75,10333222,1.092975e+10,158659,21.402,18.00,0,0,0
78,2020-03-31,853.8,873.60,838.0,857.95,861.90,854.73,2503.30,738.75,17605997,1.504845e+10,411360,21.022,35.60,1,0,0
837,2023-04-24,1687.0,1691.45,1674.1,1689.50,1688.15,1681.20,1720.00,1271.60,17837926,2.998912e+10,181877,20.477,17.35,1,0,0
826,2023-04-05,1618.6,1656.00,1616.0,1654.00,1653.75,1647.03,1722.10,1271.60,21682696,3.571205e+10,285701,20.060,40.00,1,0,0
852,2023-05-16,1671.8,1671.80,1644.0,1646.00,1647.30,1654.31,1734.45,1271.60,16361043,2.706632e+10,236552,19.982,27.80,0,0,0
718,2022-10-28,1450.0,1467.00,1447.4,1459.95,1458.70,1459.47,1722.10,1271.60,5237139,7.643453e+09,142985,21.258,19.60,1,0,0
798,2023-02-21,1640.0,1662.35,1634.0,1649.95,1646.50,1647.50,1722.10,1271.60,6194648,1.020567e+10,167166,23.994,28.35,1,0,0
17,2020-01-24,1246.0,1254.00,1239.1,1245.80,1244.55,1247.28,2503.30,1084.00,5915878,7.378771e+09,138884,30.355,14.90,0,0,0
500,2021-12-13,1526.0,1537.65,1507.2,1513.20,1511.15,1523.63,1725.00,1342.00,5161071,7.863563e+09,143473,26.172,30.45,0,0,0


In [8]:
((stock_df[['IsGreen', 'Is52WLow', 'Is52WHigh']].sum() * 100) / len(stock_df)).round(2)

IsGreen      51.60
Is52WLow      1.64
Is52WHigh     2.37
dtype: float64

In [9]:
stock_df.shape

(1097, 17)

### Streak

In [10]:
stock_df["Streak"] = (stock_df["IsGreen"] != stock_df["IsGreen"].shift(1)).cumsum()
stock_df["Streak"] = stock_df.groupby("Streak").cumcount() + 1
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak
0,2020-01-01,1276.10,1280.00,1270.60,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,31.185,9.40,1,0,0,1
1,2020-01-02,1279.00,1288.00,1279.00,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,31.384,9.00,1,0,0,2
2,2020-01-03,1282.20,1285.00,1263.60,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,30.937,21.40,0,0,0,1
3,2020-01-06,1260.00,1261.80,1236.00,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,30.267,25.80,0,0,0,2
4,2020-01-07,1258.90,1271.45,1252.25,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,30.746,19.20,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,2024-05-03,1533.25,1540.60,1507.20,1515.65,1519.60,1521.12,1757.5,1363.55,15932621,2.423550e+10,309946,18.433,33.40,0,0,0,1
1093,2024-05-06,1522.00,1534.50,1517.15,1523.75,1522.65,1525.73,1757.5,1363.55,14396698,2.196542e+10,290222,18.470,17.35,1,0,0,1
1094,2024-05-07,1521.95,1524.05,1504.15,1505.10,1506.15,1510.70,1757.5,1363.55,14240301,2.151282e+10,344750,18.270,19.90,0,0,0,1
1095,2024-05-08,1495.95,1496.95,1480.35,1483.50,1482.65,1486.59,1757.5,1363.55,20843931,3.098642e+10,465848,17.985,16.60,0,0,0,2


### Date based columns

In [11]:
stock_df['Day'] = stock_df['Date'].dt.day
stock_df['Month'] = stock_df['Date'].dt.month
stock_df['Year'] = stock_df['Date'].dt.year
stock_df['Weekday'] = stock_df['Date'].dt.weekday
stock_df['DayOfYear'] = stock_df['Date'].dt.day_of_year
stock_df['Quarter'] = stock_df['Date'].dt.quarter
stock_df['DaysSinceLastTradingSession'] = (stock_df['Date'] - stock_df['Date'].shift(1)).dt.days.fillna(1).astype(int)

stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession
1006,2023-12-27,1681.50,1706.50,1678.60,1702.10,1703.30,1695.87,1757.5,1460.25,13504539,2.290188e+10,321373,20.661,27.90,1,0,0,2,27,12,2023,2,361,4,1
988,2023-11-30,1569.15,1572.45,1542.15,1556.60,1558.80,1555.55,1757.5,1460.25,34745720,5.404869e+10,352886,18.908,30.30,0,0,0,1,30,11,2023,3,334,4,1
945,2023-09-27,1523.00,1532.50,1515.00,1527.45,1526.85,1524.39,1757.5,1365.00,19669269,2.998360e+10,384553,18.521,17.50,0,0,0,1,27,9,2023,2,270,3,1
177,2020-08-25,1119.00,1127.00,1106.20,1120.75,1119.70,1116.31,2288.8,738.75,13800972,1.540611e+10,206191,22.529,20.80,1,0,0,3,25,8,2020,1,238,3,1
443,2021-09-20,1564.00,1581.70,1558.00,1559.75,1559.85,1569.06,1641.0,1025.00,4390805,6.889437e+09,144006,27.015,23.70,0,0,0,1,20,9,2021,0,263,3,3
109,2020-05-20,836.45,864.00,832.20,863.50,857.10,844.42,2503.3,738.75,20007413,1.689474e+10,359835,17.245,31.80,1,0,0,1,20,5,2020,2,141,2,1
73,2020-03-24,795.25,810.00,738.75,774.80,767.70,775.82,2503.3,738.75,30528676,2.368471e+10,634897,18.724,71.25,0,1,0,3,24,3,2020,1,84,1,1
453,2021-10-04,1589.00,1601.35,1583.60,1588.00,1585.75,1591.34,1641.0,1111.00,3525514,5.610301e+09,141789,27.464,17.75,1,0,0,1,4,10,2021,0,277,4,3
1035,2024-02-07,1453.00,1453.80,1426.65,1430.00,1429.95,1435.86,1757.5,1380.25,27418543,3.936909e+10,523984,17.345,27.15,0,0,0,4,7,2,2024,2,38,1,1
347,2021-05-03,1393.00,1421.90,1377.30,1419.00,1414.45,1398.07,1641.0,826.10,11236850,1.570995e+10,341238,24.497,44.60,1,0,0,1,3,5,2021,0,123,2,3


In [12]:
stock_df.shape

(1097, 25)

### Moving averages

#### `Close`

In [13]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Close {window}MA'] = stock_df['Close'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Close.*")

,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA
0,1278.60,1278.60,1278.60,1278.60,1278.60
1,1286.75,1282.68,1282.68,1282.68,1282.68
2,1268.40,1277.92,1277.92,1277.92,1277.92
3,1240.95,1268.68,1268.68,1268.68,1268.68
4,1260.60,1267.06,1267.06,1267.06,1267.06
...,...,...,...,...,...
1092,1519.60,1519.10,1515.92,1495.77,1461.44
1093,1522.65,1520.66,1515.00,1498.32,1462.74
1094,1506.15,1520.01,1514.15,1500.22,1463.77
1095,1482.65,1516.13,1513.35,1501.94,1464.65


#### `Range`

In [14]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Range {window}MA'] = stock_df['Range'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Range.*")

,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA
0,9.40,9.40,9.40,9.40,9.40
1,9.00,9.20,9.20,9.20,9.20
2,21.40,13.27,13.27,13.27,13.27
3,25.80,16.40,16.40,16.40,16.40
4,19.20,16.96,16.96,16.96,16.96
...,...,...,...,...,...
1092,33.40,21.59,25.86,23.17,22.70
1093,17.35,22.26,25.89,23.07,22.69
1094,19.90,23.18,26.12,23.17,22.74
1095,16.60,22.34,26.03,22.82,22.56


#### `VWAP`

In [15]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'VWAP {window}MA'] = stock_df['VWAP'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "VWAP.*")

,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA
0,1276.64,1276.64,1276.64,1276.64,1276.64
1,1284.56,1280.60,1280.60,1280.60,1280.60
2,1270.48,1277.23,1277.23,1277.23,1277.23
3,1247.24,1269.73,1269.73,1269.73,1269.73
4,1261.48,1268.08,1268.08,1268.08,1268.08
...,...,...,...,...,...
1092,1521.12,1520.64,1516.89,1496.16,1461.38
1093,1525.73,1522.24,1515.98,1498.77,1462.77
1094,1510.70,1521.97,1515.35,1500.94,1463.91
1095,1486.59,1517.88,1514.38,1502.63,1464.76


#### `Volume`

In [16]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Volume {window}MA'] = stock_df['Volume'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Volume.*")

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1836849,1836849,1836849,1836849,1836849
1,3068583,2452716,2452716,2452716,2452716
2,5427775,3444402,3444402,3444402,3444402
3,5445093,3944575,3944575,3944575,3944575
4,7362247,4628109,4628109,4628109,4628109
...,...,...,...,...,...
1092,15932621,16891604,16249033,19009577,21549946
1093,14396698,17345305,16281900,19132169,21468183
1094,14240301,16847833,15861780,19003786,21363223
1095,20843931,18050202,16470867,18971816,21253646


#### `Value`

In [17]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Value {window}MA'] = stock_df['Value'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Value.*")

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,2.345001e+09,2345000988,2345000988,2345000988,2345000988
1,3.941792e+09,3143396262,3143396262,3143396262,3143396262
2,6.895886e+09,4394226092,4394226092,4394226092,4394226092
3,6.791348e+09,4993506527,4993506527,4993506527,4993506527
4,9.287302e+09,5852265530,5852265530,5852265530,5852265530
...,...,...,...,...,...
1092,2.423550e+10,25703390202,24667407027,28388441305,31387829153
1093,2.196542e+10,26413613564,24704864940,28603455514,31289989988
1094,2.151282e+10,25657160154,24057270993,28448687103,31154904648
1095,3.098642e+10,27393821809,24951340955,28438063101,31015193488


#### `#Trades`

In [18]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'#Trades {window}MA'] = stock_df['#Trades'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "#Trades.*")

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,46625,46625,46625,46625,46625
1,104570,75597,75597,75597,75597
2,157066,102753,102753,102753,102753
3,155007,115817,115817,115817,115817
4,189026,130458,130458,130458,130458
...,...,...,...,...,...
1092,309946,344552,346571,361159,403849
1093,290222,347342,348256,360885,400569
1094,344750,337695,347642,360460,399469
1095,465848,360309,358449,362926,398500


### Target columns

In [19]:
for window in cnst.TARGET_WINDOWS:
    stock_df[f'Target {window}D'] = stock_df['Close'].shift(-window)

stock_df.filter(regex = "Target.*")

,Target 3D,Target 7D,Target 15D,Target 30D
0,1240.95,1282.70,1240.85,1240.60
1,1260.60,1286.00,1244.85,1249.00
2,1257.30,1289.50,1244.55,1241.40
3,1271.40,1284.25,1213.20,1219.35
4,1282.70,1287.65,1223.20,1217.15
...,...,...,...,...
1092,1482.65,NaN,NaN,NaN
1093,1447.50,NaN,NaN,NaN
1094,NaN,NaN,NaN,NaN
1095,NaN,NaN,NaN,NaN


### Saving processed data

In [20]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-processed.parquet'), 
    index = False
)
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,PE,Range,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,2020-01-01,1276.10,1280.00,1270.60,1279.00,1278.60,1276.64,2503.3,1084.00,1836849,2.345001e+09,46625,31.185,9.40,1,0,0,1,1,1,2020,2,1,1,1,1278.60,1278.60,1278.60,1278.60,9.40,9.40,9.40,9.40,1276.64,1276.64,1276.64,1276.64,1836849,1836849,1836849,1836849,2345000988,2345000988,2345000988,2345000988,46625,46625,46625,46625,1240.95,1282.70,1240.85,1240.60
1,2020-01-02,1279.00,1288.00,1279.00,1286.00,1286.75,1284.56,2503.3,1084.00,3068583,3.941792e+09,104570,31.384,9.00,1,0,0,2,2,1,2020,3,2,1,1,1282.68,1282.68,1282.68,1282.68,9.20,9.20,9.20,9.20,1280.60,1280.60,1280.60,1280.60,2452716,2452716,2452716,2452716,3143396262,3143396262,3143396262,3143396262,75597,75597,75597,75597,1260.60,1286.00,1244.85,1249.00
2,2020-01-03,1282.20,1285.00,1263.60,1268.50,1268.40,1270.48,2503.3,1084.00,5427775,6.895886e+09,157066,30.937,21.40,0,0,0,1,3,1,2020,4,3,1,1,1277.92,1277.92,1277.92,1277.92,13.27,13.27,13.27,13.27,1277.23,1277.23,1277.23,1277.23,3444402,3444402,3444402,3444402,4394226092,4394226092,4394226092,4394226092,102753,102753,102753,102753,1257.30,1289.50,1244.55,1241.40
3,2020-01-06,1260.00,1261.80,1236.00,1240.25,1240.95,1247.24,2503.3,1084.00,5445093,6.791348e+09,155007,30.267,25.80,0,0,0,2,6,1,2020,0,6,1,3,1268.68,1268.68,1268.68,1268.68,16.40,16.40,16.40,16.40,1269.73,1269.73,1269.73,1269.73,3944575,3944575,3944575,3944575,4993506527,4993506527,4993506527,4993506527,115817,115817,115817,115817,1271.40,1284.25,1213.20,1219.35
4,2020-01-07,1258.90,1271.45,1252.25,1261.00,1260.60,1261.48,2503.3,1084.00,7362247,9.287302e+09,189026,30.746,19.20,1,0,0,1,7,1,2020,1,7,1,1,1267.06,1267.06,1267.06,1267.06,16.96,16.96,16.96,16.96,1268.08,1268.08,1268.08,1268.08,4628109,4628109,4628109,4628109,5852265530,5852265530,5852265530,5852265530,130458,130458,130458,130458,1282.70,1287.65,1223.20,1217.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,2024-05-03,1533.25,1540.60,1507.20,1515.65,1519.60,1521.12,1757.5,1363.55,15932621,2.423550e+10,309946,18.433,33.40,0,0,0,1,3,5,2024,4,124,2,1,1519.10,1515.92,1495.77,1461.44,21.59,25.86,23.17,22.70,1520.64,1516.89,1496.16,1461.38,16891604,16249033,19009577,21549946,25703390202,24667407027,28388441305,31387829153,344552,346571,361159,403849,1482.65,NaN,NaN,NaN
1093,2024-05-06,1522.00,1534.50,1517.15,1523.75,1522.65,1525.73,1757.5,1363.55,14396698,2.196542e+10,290222,18.470,17.35,1,0,0,1,6,5,2024,0,127,2,3,1520.66,1515.00,1498.32,1462.74,22.26,25.89,23.07,22.69,1522.24,1515.98,1498.77,1462.77,17345305,16281900,19132169,21468183,26413613564,24704864940,28603455514,31289989988,347342,348256,360885,400569,1447.50,NaN,NaN,NaN
1094,2024-05-07,1521.95,1524.05,1504.15,1505.10,1506.15,1510.70,1757.5,1363.55,14240301,2.151282e+10,344750,18.270,19.90,0,0,0,1,7,5,2024,1,128,2,1,1520.01,1514.15,1500.22,1463.77,23.18,26.12,23.17,22.74,1521.97,1515.35,1500.94,1463.91,16847833,15861780,19003786,21363223,25657160154,24057270993,28448687103,31154904648,337695,347642,360460,399469,NaN,NaN,NaN,NaN
1095,2024-05-08,1495.95,1496.95,1480.35,1483.50,1482.65,1486.59,1757.5,1363.55,20843931,3.098642e+10,465848,17.985,16.60,0,0,0,2,8,5,2024,2,129,2,1,1516.13,1513.35,1501.94,1464.65,22.34,26.03,22.82,22.56,1517.88,1514.38,1502.63,1464.76,18050202,16470867,18971816,21253646,27393821809,24951340955,28438063101,31015193488,360309,358449,362926,398500,NaN,NaN,NaN,NaN


## Data standardization

### Stock price based columns

In [21]:
stock_price_cols = [
    'Open', 'High', 'Low', 'LTP', '52W H', '52W L'
] + (
    stock_df.filter(regex = "Close.*").columns.to_list() +
    stock_df.filter(regex = "Range.*").columns.to_list() +
    stock_df.filter(regex = "VWAP.*").columns.to_list() +
    stock_df.filter(regex = "Target.*").columns.to_list()
)

stock_price_cols

['Open',
 'High',
 'Low',
 'LTP',
 '52W H',
 '52W L',
 'Close',
 'Close 7MA',
 'Close 15MA',
 'Close 30MA',
 'Close 60MA',
 'Range',
 'Range 7MA',
 'Range 15MA',
 'Range 30MA',
 'Range 60MA',
 'VWAP',
 'VWAP 7MA',
 'VWAP 15MA',
 'VWAP 30MA',
 'VWAP 60MA',
 'Target 3D',
 'Target 7D',
 'Target 15D',
 'Target 30D']

In [22]:
stock_df[stock_price_cols] = stock_df[stock_price_cols].div(stock_df['Close'], axis = 0).round(3)
stock_df[stock_price_cols]

,Open,High,Low,LTP,52W H,52W L,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,1.958,0.848,1.0,1.000,1.000,1.000,1.000,0.007,0.007,0.007,0.007,0.007,0.998,0.998,0.998,0.998,0.998,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,1.945,0.842,1.0,0.997,0.997,0.997,0.997,0.007,0.007,0.007,0.007,0.007,0.998,0.995,0.995,0.995,0.995,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.974,0.855,1.0,1.008,1.008,1.008,1.008,0.017,0.010,0.010,0.010,0.010,1.002,1.007,1.007,1.007,1.007,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,2.017,0.874,1.0,1.022,1.022,1.022,1.022,0.021,0.013,0.013,0.013,0.013,1.005,1.023,1.023,1.023,1.023,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.986,0.860,1.0,1.005,1.005,1.005,1.005,0.015,0.013,0.013,0.013,0.013,1.001,1.006,1.006,1.006,1.006,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,1.009,1.014,0.992,0.997,1.157,0.897,1.0,1.000,0.998,0.984,0.962,0.022,0.014,0.017,0.015,0.015,1.001,1.001,0.998,0.985,0.962,0.976,NaN,NaN,NaN
1093,1.000,1.008,0.996,1.001,1.154,0.896,1.0,0.999,0.995,0.984,0.961,0.011,0.015,0.017,0.015,0.015,1.002,1.000,0.996,0.984,0.961,0.951,NaN,NaN,NaN
1094,1.010,1.012,0.999,0.999,1.167,0.905,1.0,1.009,1.005,0.996,0.972,0.013,0.015,0.017,0.015,0.015,1.003,1.011,1.006,0.997,0.972,NaN,NaN,NaN,NaN
1095,1.009,1.010,0.998,1.001,1.185,0.920,1.0,1.023,1.021,1.013,0.988,0.011,0.015,0.018,0.015,0.015,1.003,1.024,1.021,1.013,0.988,NaN,NaN,NaN,NaN


### Volume based columns

In [23]:
volume_cols = stock_df.filter(regex = "Volume.*").columns.to_list()

volume_cols

['Volume', 'Volume 7MA', 'Volume 15MA', 'Volume 30MA', 'Volume 60MA']

In [24]:
stock_df[volume_cols] = stock_df[volume_cols].div(stock_df['Volume'], axis = 0).round(3)
stock_df[volume_cols]

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.799,0.799,0.799,0.799
2,1.0,0.635,0.635,0.635,0.635
3,1.0,0.724,0.724,0.724,0.724
4,1.0,0.629,0.629,0.629,0.629
...,...,...,...,...,...
1092,1.0,1.060,1.020,1.193,1.353
1093,1.0,1.205,1.131,1.329,1.491
1094,1.0,1.183,1.114,1.335,1.500
1095,1.0,0.866,0.790,0.910,1.020


### Value based columns

In [25]:
value_cols = stock_df.filter(regex = "Value.*").columns.to_list()

value_cols

['Value', 'Value 7MA', 'Value 15MA', 'Value 30MA', 'Value 60MA']

In [26]:
stock_df[value_cols] = stock_df[value_cols].div(stock_df['Value'], axis = 0).round(3)
stock_df[value_cols]

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.797,0.797,0.797,0.797
2,1.0,0.637,0.637,0.637,0.637
3,1.0,0.735,0.735,0.735,0.735
4,1.0,0.630,0.630,0.630,0.630
...,...,...,...,...,...
1092,1.0,1.061,1.018,1.171,1.295
1093,1.0,1.203,1.125,1.302,1.425
1094,1.0,1.193,1.118,1.322,1.448
1095,1.0,0.884,0.805,0.918,1.001


### Trade count based columns

In [27]:
trade_count_cols = stock_df.filter(regex = "#Trades.*").columns.to_list()

trade_count_cols

['#Trades', '#Trades 7MA', '#Trades 15MA', '#Trades 30MA', '#Trades 60MA']

In [28]:
stock_df[trade_count_cols] = stock_df[trade_count_cols].div(stock_df['#Trades'], axis = 0).round(3)
stock_df[trade_count_cols]

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,0.723,0.723,0.723,0.723
2,1.0,0.654,0.654,0.654,0.654
3,1.0,0.747,0.747,0.747,0.747
4,1.0,0.690,0.690,0.690,0.690
...,...,...,...,...,...
1092,1.0,1.112,1.118,1.165,1.303
1093,1.0,1.197,1.200,1.243,1.380
1094,1.0,0.980,1.008,1.046,1.159
1095,1.0,0.773,0.769,0.779,0.855


### Dropping unnecessary columns

In [29]:
stock_df = stock_df.drop(columns = ['Date', 'Close', 'Range', 'Volume', 'Value', '#Trades'])
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,PE,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,0.998,1.958,0.848,31.185,1,0,0,1,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.007,0.007,0.007,0.007,0.998,0.998,0.998,0.998,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,0.998,1.945,0.842,31.384,1,0,0,2,2,1,2020,3,2,1,1,0.997,0.997,0.997,0.997,0.007,0.007,0.007,0.007,0.995,0.995,0.995,0.995,0.799,0.799,0.799,0.799,0.797,0.797,0.797,0.797,0.723,0.723,0.723,0.723,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.002,1.974,0.855,30.937,0,0,0,1,3,1,2020,4,3,1,1,1.008,1.008,1.008,1.008,0.010,0.010,0.010,0.010,1.007,1.007,1.007,1.007,0.635,0.635,0.635,0.635,0.637,0.637,0.637,0.637,0.654,0.654,0.654,0.654,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,1.005,2.017,0.874,30.267,0,0,0,2,6,1,2020,0,6,1,3,1.022,1.022,1.022,1.022,0.013,0.013,0.013,0.013,1.023,1.023,1.023,1.023,0.724,0.724,0.724,0.724,0.735,0.735,0.735,0.735,0.747,0.747,0.747,0.747,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.001,1.986,0.860,30.746,1,0,0,1,7,1,2020,1,7,1,1,1.005,1.005,1.005,1.005,0.013,0.013,0.013,0.013,1.006,1.006,1.006,1.006,0.629,0.629,0.629,0.629,0.630,0.630,0.630,0.630,0.690,0.690,0.690,0.690,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,1.009,1.014,0.992,0.997,1.001,1.157,0.897,18.433,0,0,0,1,3,5,2024,4,124,2,1,1.000,0.998,0.984,0.962,0.014,0.017,0.015,0.015,1.001,0.998,0.985,0.962,1.060,1.020,1.193,1.353,1.061,1.018,1.171,1.295,1.112,1.118,1.165,1.303,0.976,NaN,NaN,NaN
1093,1.000,1.008,0.996,1.001,1.002,1.154,0.896,18.470,1,0,0,1,6,5,2024,0,127,2,3,0.999,0.995,0.984,0.961,0.015,0.017,0.015,0.015,1.000,0.996,0.984,0.961,1.205,1.131,1.329,1.491,1.203,1.125,1.302,1.425,1.197,1.200,1.243,1.380,0.951,NaN,NaN,NaN
1094,1.010,1.012,0.999,0.999,1.003,1.167,0.905,18.270,0,0,0,1,7,5,2024,1,128,2,1,1.009,1.005,0.996,0.972,0.015,0.017,0.015,0.015,1.011,1.006,0.997,0.972,1.183,1.114,1.335,1.500,1.193,1.118,1.322,1.448,0.980,1.008,1.046,1.159,NaN,NaN,NaN,NaN
1095,1.009,1.010,0.998,1.001,1.003,1.185,0.920,17.985,0,0,0,2,8,5,2024,2,129,2,1,1.023,1.021,1.013,0.988,0.015,0.018,0.015,0.015,1.024,1.021,1.013,0.988,0.866,0.790,0.910,1.020,0.884,0.805,0.918,1.001,0.773,0.769,0.779,0.855,NaN,NaN,NaN,NaN


### Saving standardized data

In [30]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-standardized.parquet'), 
    index = False
)
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,PE,IsGreen,Is52WLow,Is52WHigh,Streak,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.998,1.001,0.994,1.000,0.998,1.958,0.848,31.185,1,0,0,1,1,1,2020,2,1,1,1,1.000,1.000,1.000,1.000,0.007,0.007,0.007,0.007,0.998,0.998,0.998,0.998,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.971,1.003,0.970,0.970
1,0.994,1.001,0.994,0.999,0.998,1.945,0.842,31.384,1,0,0,2,2,1,2020,3,2,1,1,0.997,0.997,0.997,0.997,0.007,0.007,0.007,0.007,0.995,0.995,0.995,0.995,0.799,0.799,0.799,0.799,0.797,0.797,0.797,0.797,0.723,0.723,0.723,0.723,0.980,0.999,0.967,0.971
2,1.011,1.013,0.996,1.000,1.002,1.974,0.855,30.937,0,0,0,1,3,1,2020,4,3,1,1,1.008,1.008,1.008,1.008,0.010,0.010,0.010,0.010,1.007,1.007,1.007,1.007,0.635,0.635,0.635,0.635,0.637,0.637,0.637,0.637,0.654,0.654,0.654,0.654,0.991,1.017,0.981,0.979
3,1.015,1.017,0.996,0.999,1.005,2.017,0.874,30.267,0,0,0,2,6,1,2020,0,6,1,3,1.022,1.022,1.022,1.022,0.013,0.013,0.013,0.013,1.023,1.023,1.023,1.023,0.724,0.724,0.724,0.724,0.735,0.735,0.735,0.735,0.747,0.747,0.747,0.747,1.025,1.035,0.978,0.983
4,0.999,1.009,0.993,1.000,1.001,1.986,0.860,30.746,1,0,0,1,7,1,2020,1,7,1,1,1.005,1.005,1.005,1.005,0.013,0.013,0.013,0.013,1.006,1.006,1.006,1.006,0.629,0.629,0.629,0.629,0.630,0.630,0.630,0.630,0.690,0.690,0.690,0.690,1.018,1.021,0.970,0.966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,1.009,1.014,0.992,0.997,1.001,1.157,0.897,18.433,0,0,0,1,3,5,2024,4,124,2,1,1.000,0.998,0.984,0.962,0.014,0.017,0.015,0.015,1.001,0.998,0.985,0.962,1.060,1.020,1.193,1.353,1.061,1.018,1.171,1.295,1.112,1.118,1.165,1.303,0.976,NaN,NaN,NaN
1093,1.000,1.008,0.996,1.001,1.002,1.154,0.896,18.470,1,0,0,1,6,5,2024,0,127,2,3,0.999,0.995,0.984,0.961,0.015,0.017,0.015,0.015,1.000,0.996,0.984,0.961,1.205,1.131,1.329,1.491,1.203,1.125,1.302,1.425,1.197,1.200,1.243,1.380,0.951,NaN,NaN,NaN
1094,1.010,1.012,0.999,0.999,1.003,1.167,0.905,18.270,0,0,0,1,7,5,2024,1,128,2,1,1.009,1.005,0.996,0.972,0.015,0.017,0.015,0.015,1.011,1.006,0.997,0.972,1.183,1.114,1.335,1.500,1.193,1.118,1.322,1.448,0.980,1.008,1.046,1.159,NaN,NaN,NaN,NaN
1095,1.009,1.010,0.998,1.001,1.003,1.185,0.920,17.985,0,0,0,2,8,5,2024,2,129,2,1,1.023,1.021,1.013,0.988,0.015,0.018,0.015,0.015,1.024,1.021,1.013,0.988,0.866,0.790,0.910,1.020,0.884,0.805,0.918,1.001,0.773,0.769,0.779,0.855,NaN,NaN,NaN,NaN
